In [1]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import os
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from tqdm import tqdm

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

In [2]:
"""INDEX 1"""
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)]
"""INDEX 1"""

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

In [4]:
page_num = 100
pix_map = doc.get_page_pixmap(page_num,matrix=mat)
image = Image.open(io.BytesIO(pix_map.tobytes()))
draw = ImageDraw.Draw(image)
image;

block_lst = pages[page_num].get_text("dict")['blocks']
block_lst[0]['lines'][0]['spans']

blank_im = Image.new('RGB', image.size, (255, 255, 255))
blank_draw = ImageDraw.Draw(blank_im)
dy_block_lst = []
x0, y0, x1, y1 = 0, 0, 0, 0
myFont = ImageFont.truetype("Arial", 15)
g_i = 0
#block_lst = dict(block_lst)

for b in block_lst:
    if b['type'] == 0:
        x0_b, y0_b, x1_, y1_b = [coord*TARGET_DPI/ 72 for coord in b['bbox']]
        draw.rectangle((x0_b, y0_b, x1_, y1_b), fill=None, outline=ImageColor.getrgb("#8100bb"), width = 1)
        blank_draw.rectangle((x0_b, y0_b, x1_, y1_b), fill=None, outline=ImageColor.getrgb("#8100bb"), width = 1)
        for l in b['lines']:
            #for s in l['spans']:
            #    for w in s['text']:
            prev_x0, prev_y0, prev_x1, prev_y1 = x0, y0, x1, y1
            x0, y0, x1, y1 = [coord*TARGET_DPI/ 72 for coord in l['bbox']]
            #print(y0, prev_y1)
            w = 1 
            if x0 > 300 and x1 < 1400: 
                w = 5
            draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#222222"), width = w)
            dy = y0 - prev_y1
            dy_block_lst.append(dy)
            blank_draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb("#222222"), width = w)
            blank_draw.text((20, prev_y1), str(dy), font=myFont, fill =(255, 0, 0))
            draw.text((20, prev_y1), str(dy), font=myFont, fill =(255, 0, 0))
            
            l['line_group'] = g_i
            #print(dy)
            if prev_y1 == 0 or dy > 15:
                g_i += 1

            blank_draw.text((prev_x1, prev_y1), str(l['line_group']), font=myFont, fill =(0, 0, 255))
            draw.text((prev_x1, prev_y1), str(l['line_group']), font=myFont, fill =(0, 0, 255))